In [1]:
from tinygrad.tensor import Tensor
(Tensor.ones(4,4).gpu() + Tensor.ones(4,4).gpu()).cpu()



<Tensor array([[2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.]], dtype=float32) with grad None>

In [1]:
import tensorflow as tf1
from tensorflow.python.compiler.mlcompute import mlcompute
mlcompute.is_tf_compiled_with_apple_mlc()

from tensorflow.python.framework.ops import disable_eager_execution

import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

to_categorical = tf.keras.utils.to_categorical
fashion_mnist = tf.keras.datasets.fashion_mnist

import time

mlcompute.set_mlc_device(device_name='GPU')
tf.executing_eagerly()
batch_size = 128
num_classes = 10
epochs = 5

img_rows, img_cols = 28, 28
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train.shape, x_test.shape
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')



x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [2]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
model = tf.keras.Sequential([ 
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=input_shape),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation="softmax")
])
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [4]:
start = time.time()
model.fit(x_train, y_train, batch_size=batch_size, epochs=5, verbose=1, validation_split=0.2, shuffle= True)
done = time.time()

Epoch 1/5
375/375 [==============================] - 18s 48ms/step - loss: 1.5916 - accuracy: 0.5270 - val_loss: 0.5309 - val_accuracy: 0.8105
Epoch 2/5
375/375 [==============================] - 17s 46ms/step - loss: 0.5370 - accuracy: 0.8113 - val_loss: 0.4139 - val_accuracy: 0.8523
Epoch 3/5
375/375 [==============================] - 17s 45ms/step - loss: 0.4294 - accuracy: 0.8492 - val_loss: 0.3471 - val_accuracy: 0.8733
Epoch 4/5
375/375 [==============================] - 17s 46ms/step - loss: 0.3794 - accuracy: 0.8647 - val_loss: 0.3298 - val_accuracy: 0.8810
Epoch 5/5
375/375 [==============================] - 16s 44ms/step - loss: 0.3467 - accuracy: 0.8777 - val_loss: 0.3071 - val_accuracy: 0.8857


In [3]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices())



['/device:CPU:0']


In [4]:
import sys
print (sys.version)
# 3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
import tensorflow as tf
print(tf.__version__)

3.8.6 | packaged by conda-forge | (default, Jan 25 2021, 22:55:00) 
[Clang 11.0.1 ]
2.4.0-rc0


In [5]:
import tensorflow as tf
from tensorflow import keras
print("tf version {}".format(tf.__version__))
if tf.config.list_physical_devices('GPU'):
    print(tf.test.gpu_device_name())
else:
    print("TF cannot find GPU")

tf version 2.4.0-rc0
TF cannot find GPU


In [6]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   print('GPU device not found')
print('Found GPU at: {}'.format(device_name))

GPU device not found
Found GPU at: 


In [7]:
from tensorflow.python.compiler.mlcompute import mlcompute
import tensorflow as tf 
from tensorflow.python.framework.ops import disable_eager_execution 
disable_eager_execution() 
print(tf.executing_eagerly())

False


In [8]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
if physical_devices:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

[]


In [9]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0



model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)
])
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
history = model.fit(
    train_images, 
    train_labels, 
    epochs=10, 
    validation_data=(test_images, test_labels)
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
59872/60000 [============================>.] - ETA: 0s - loss: 0.2387 - accuracy: 0.9295

/Users/elahe/miniforge3/envs/tf24/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


60000/60000 [==============================] - 7s 114us/sample - loss: 0.2384 - accuracy: 0.9295 - val_loss: 0.1381 - val_accuracy: 0.9566
Epoch 2/10
60000/60000 [==============================] - 6s 106us/sample - loss: 0.1009 - accuracy: 0.9691 - val_loss: 0.0912 - val_accuracy: 0.9717
Epoch 3/10
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0694 - accuracy: 0.9784 - val_loss: 0.0883 - val_accuracy: 0.9733
Epoch 4/10
60000/60000 [==============================] - 8s 137us/sample - loss: 0.0532 - accuracy: 0.9831 - val_loss: 0.0802 - val_accuracy: 0.9771
Epoch 5/10
60000/60000 [==============================] - 10s 171us/sample - loss: 0.0396 - accuracy: 0.9873 - val_loss: 0.0751 - val_accuracy: 0.9777
Epoch 6/10
60000/60000 [==============================] - 10s 168us/sample - loss: 0.0346 - accuracy: 0.9884 - val_loss: 0.0824 - val_accuracy: 0.9771
Epoch 7/10
60000/60000 [==============================] - 10s 164us/sample - loss: 0.0288 - accuracy: 0.9906 

In [10]:

import tensorflow as tf
from tensorflow.keras import datasets, layers, models

(train_images, train_labels), (test_images, test_labels) = datasets.fashion_mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0



model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)
])
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
history = model.fit(
    train_images, 
    train_labels, 
    epochs=100, 
    validation_data=(test_images, test_labels)
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 7s 124us/sample - loss: 0.4971 - accuracy: 0.8218 - val_loss: 0.4095 - val_accuracy: 0.8532
Epoch 2/100
60000/60000 [==============================] - 8s 125us/sample - loss: 0.3670 - accuracy: 0.8656 - val_loss: 0.3935 - val_accuracy: 0.8602
Epoch 3/100
60000/60000 [==============================] - 9s 152us/sample - loss: 0.3342 - accuracy: 0.8774 - val_loss: 0.3678 - val_accuracy: 0.8672
Epoch 4/100
60000/60000 [==============================] - 8s 136us/sample - loss: 0.3088 - accuracy: 0.8861 - val_loss: 0.3763 - val_accuracy: 0.8640
Epoch 5/100
60000/60000 [==============================] - 10s 159us/sample - loss: 0.2898 - accuracy: 0.8923 - val_loss: 0.3499 - val_accuracy: 0.8766
Epoch 6/100
60000/60000 [==============================] - 9s 149us/sample - loss: 0.2770 - accuracy: 0.8962 - val_loss: 0.3393 - val_accuracy: 0.8787
Epoch 7/100
60000/60000 [==================